In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate

In [2]:
import datasets

dataset = datasets.load_dataset("csv", data_files="Datasets/train.csv")
dataset_val = datasets.load_dataset("csv", data_files="Datasets/val.csv")
dataset_test = datasets.load_dataset("csv", data_files="Datasets/test.csv")
dataset["val"] = dataset_val["train"]
dataset["test"] = dataset_test["train"]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 40428
    })
})

In [4]:
dataset["val"].features

{'question1': Value(dtype='string', id=None),
 'question2': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None)}

In [5]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# change model to finetune here
checkpoint = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(data):
    return tokenizer(data["question1"], data["question2"], truncation = True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)
tokenized_dataset

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-05-16 16:48:28.546617: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-16 16:48:28.577921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 16:48:28.577950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 16:48:28.578855: E external/local_xla/xla/stream_executor/cuda/c

Map:   0%|          | 0/40428 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
})

In [6]:
tokenized_dataset = tokenized_dataset.remove_columns(["question1", "question2"])
tokenized_dataset = tokenized_dataset.with_format("torch")
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
})

In [7]:
small_train_dataset = tokenized_dataset["train"].select(range(10000))
small_train_dataset
small_val_dataset = tokenized_dataset["val"].select(range(10000))
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [8]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
import torch


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer2", 
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs = 10
                                 )

In [19]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
metric2 = evaluate.load("confusion_matrix")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)#", metric2.compute(predictions=predictions, references=labels)


In [20]:
small_train_dataset = tokenized_dataset["train"].select(range(10000))
small_train_dataset
small_val_dataset = tokenized_dataset["val"].select(range(10000))
small_train_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset if False else tokenized_dataset["train"],
    eval_dataset=small_val_dataset if False else tokenized_dataset["val"] ,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [16]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.292900,0.271707,0.881246
2,0.231900,0.290526,0.894974
3,0.190900,0.308282,0.899525
4,0.156500,0.342982,0.902815


KeyboardInterrupt: 

In [17]:
trainer.train(True)

Epoch,Training Loss,Validation Loss,Accuracy
5,0.123900,0.394740,0.904151


TrainOutput(global_step=101075, training_loss=0.021362865239477617, metrics={'train_runtime': 891.8745, 'train_samples_per_second': 1813.209, 'train_steps_per_second': 113.329, 'total_flos': 6260826683153796.0, 'train_loss': 0.021362865239477617, 'epoch': 5.0})

In [21]:
# resume training from latest checkpoint

trainer.train(True)

Epoch,Training Loss,Validation Loss,Accuracy
5,0.123900,0.391217,0.903211
6,0.156200,0.390240,0.902394
7,0.132500,0.416112,0.903310


KeyboardInterrupt: 